# Countdown Numbers Game

***

## Example & Explanation
Example and Explanation taking from https://wiki.apterous.org/Numbers_game
***
The Countdown numbers Game is a popular televised gameshow game, this notebook is a replication of the technical and logical side of countdown numbers game. The countdown numbers game involves a board of 24 numbered cards, arranged faced down. The top row contains the large numbers which are 25, 50, 75, 100. The three other rows contain cards with small numbers between 1 and 10 inclusive. A contestant most chose six cards from the 24 cards available to them, the contestant has the option to choose the amount of "large" or "small" numbered cards, for example they could chose 1 large numbered card and 5 small numbered cards (75, 3, 2, 1, 7, 4). Once the 6 cards are chosen by the contestant, a random 3-digit number is generated called the target number, the target can range from 101 to 999. The contestant has 30 seconds to use the 6 numbers they chose to make the target number or as near as possible.    

#### Game Example
Six Chosen Numbers: 
$
\begin{align}
  75, 2, 5, 6, 1, 4
\end{align}
$ 

Random Three Digit Target Number: 
$
\begin{align}
 273
\end{align}
$ 

Solution:
$
\begin{align}
 75 × 4 = 300
\end{align}
$ 

$
\begin{align}
 5 × 6 − 2 − 1 = 27
\end{align}
$ 

$
\begin{align}
 300 − 27 = 273
\end{align}
$

The Countdown Numbers Game in this notebook follows the rules that exist in the original countdown numbers game. However, one small change is that the six numbers the player originally was able to choose himself are randomly chosen by the computer. Other than that, the game functions just like the original and follows all rules and concepts correctly.

## Overview of Countdown Numbers Game

***

### How the Game Works, Code Explanation
1. Six numbers are generate by the "generateGameNumbers()" function.

2. One Target Numbers is generated by "generateTargetNumber()" function.

3. The target number and game numbers are input into the "solveCountdownNumbersGame()" function and processed.

4. Inside the "solveCountdownNumbersGame()", Python Itertools library is used. All possible permutations of the generated numbers and cartesian product of the mathematical operators are put into a Product function that gets the cartesian product of the generated numbers and operators.

5. Each iterator of the play_nos and opers is input into the "patterns()" function, the play_nos and opers are processed and a calculation is created using Reverse Polish Notation. The Rever Polish Notation Calculation is the returned.

6. The calculation is then passed into the "eval_rpn()" where the Reverse Polish Notation calculation is evaluated and solved. The solved calculation is returned and compared to the target, if they match a correct calculation was made.

7. If a correct calculation was made the calculation and target are input into the "displayCalculation()" function to be processed and printed as a reverse polish notation for the user to understand.


In [4]:
#Imports

# Permutations and combinations.
import itertools as it

# Random number generation.
import random

# Operators as functions.
import operator

In [5]:
# Generate Random Game Numbers
def generateGameNumbers(no_large=None):
  # Returns six numbers and a target number representing a Countdown numbers game.
  
  # If no_large in None, randomly pick value between 0 and 4 inclusive.
  if no_large is None:
    # Randomly set the value.
    no_large = random.randrange(0, 5)
  
  # Select random large numbers.
  large_rand = random.sample([25, 50, 75, 100], no_large)
  # Select random small numbers.
  small_rand = random.sample(list(range(1, 11)) * 2, 6 - no_large)
  # The playing numbers.
  play_nos = large_rand + small_rand

  # Return the game.
  return play_nos

# Generate Target Number
def generateTargetNumber():
    return random.randrange(101, 1000)

In [6]:
# # Game Numbers.
gameNumbers = generateGameNumbers()

gameNumbers

[25, 100, 75, 50, 8, 6]

In [7]:
# Target Number
targetNumber = generateTargetNumber()

targetNumber

594

In [8]:
# Give all 2-partitions of a list
# where each sublist has  one element.
def patterns(numbers, operators):
  # Check if there is no way to partition further.
  if len(numbers) == 1:
    yield numbers
  # Loop through all the ways to partition L into two non-empty sublists.
  for i in range(1, len(numbers)):
    # Slice the list using i.
    for left, right in it.product(patterns(numbers[:i], operators[1:i]), patterns(numbers[i:], operators[i:])):
      # Yield the next operator applied to the sublists.
      yield [*left, *right, operators[0]]

In [9]:
# Evaluate RPN expression.
def eval_rpn(rpn):
  # A stack.
  stack = []
  # Loop through rpn an item at a time.
  for i in rpn:
    # Check if it's a number.
    if isinstance(i, int):
      # Append to the stack.
      stack = stack + [i]
    else:
      # Pop from stack twice.
      right = stack[-1]
      stack = stack[:-1]
      left = stack[-1]
      stack = stack[:-1]
      # Push operator applied to stack elements.
      #print("This is i: ", i)
      if i == operator.truediv and right == 0:
          stack = stack + [0]
      else:
          stack = stack + [i(left, right)]
  # Should only be one item on stack.
  return stack[0]

In [10]:
def displayCalculation(cal, target):
    calculation = []
    b = 0
    print("Answer: ", target, " = ", end="")
    for i in cal:
        if i == operator.truediv:
            calculation = calculation + ["/"]
        elif i == operator.sub:
            calculation = calculation + ["-"]
        elif i == operator.add:
            calculation = calculation + ["+"]
        elif i == operator.mul:
            calculation = calculation + ["*"]
        else:
            calculation = calculation + [i]
            
        print(calculation[b], " ", end="")
        b = b + 1
    
    print(" ")
    

In [17]:
# Solve Countdown Numbers Game
def solveCountdownNumbersGame(gameNumbers, targetNumber):
    print("Game Numbers: ", gameNumbers)
    print("Target: ", targetNumber)
    
    # Operators.
    ops = [operator.add, operator.sub, operator.mul, operator.truediv]
    # Limit the output.
    limit = 1100
    # For the limit.
    i = 0
    
    # Orderings of pairs.
    for play_nos, opers in it.product(it.permutations(gameNumbers), it.product(*([ops] * 5))):
      #print(play_nos, opers)
      for c in patterns(play_nos, opers):
          
          if targetNumber == eval_rpn(c): 
              displayCalculation(c, eval_rpn(c))
              break
                
      i = i + 1
      if i >= limit:
        break

In [18]:
solveCountdownNumbersGame(gameNumbers, targetNumber)

Game Numbers:  [25, 100, 75, 50, 8, 6]
Target:  594
Answer:  594  = 25  100  75  50  8  *  6  -  +  +  +   
Answer:  594  = 25  100  75  50  8  *  +  6  -  +  +   
Answer:  594  = 25  100  75  50  8  *  +  +  6  -  +   
Answer:  594  = 25  100  75  50  8  *  +  +  +  6  -   


## Complexity of Countdown Numbers Game

***

## Functional Programming

https://docs.python.org/3/howto/functional.html
***

### What is Functional Programming?
Functional programming is based on producing simplistic reduced functions that focus on an argument and return value. A function in functional programming is not meant to have side effect, you give the function an input and it will return an output without modifying the inputs. This is known as a pure function where the results of the return value are dependent on the input parameter. Functional programming allows for the creation of clean and maintainable software, keeping the data side and functionality independent, avoiding changing state of variables (it’s okay to declare a new variable instead of re-using existing variables) and treating functions as first class meaning treat functions like data, where a function can be assigned to a variable.

## Functional Aspect of Code behind Countdown Numbers Game.

The Countdown game is developed in a functional programming style, following some of the functional programming principles talked above and also using some features python has available for implementing functional style programming. Below is an explanation of each principle and python feature and how they are implemented in the Countdown Numbers Game code.

* Pure Functions: Functions in the countdown numbers game are developed to receive an input, process the input, and return an output, without interfering with any other parts of the games code, and without modifying the inputs. A good example of a pure function in the code is the "generateGameNumbers", it receives an input, process it and returns the generate numbers.

* State Of Variables: The game is developed to avoid state and tries to emphasize functionality of the functions. The variables are almost seen as immutable variables, that can’t be changed (or at least the goal is for them not to be changed). The state of variables does not change, and the value of the variables are passed through functions as arguments and are processed and the processed data is returned as a output. This can be seen in the "solveCountdownNumbersGame" function where the target and game numbers are passed in and processed and then passed into the "patterns" function and then a value is returned at the end. As you can see in those functions the state of the initial arguments was never changed.

* Iterators Python: Python Iterator feature is used to iterate over the possible different types of equation sequences. Itertools Product and permutation are used from the itertools library. Product() finds the cartesian product of the generate numbers and operators, which then the product is used to calculate an equation to solve for the target. Permutation() takes the generated numbers and creates a list of immutable tuples that contain all permutations of the generate number in a list form. This is used in the "solveCountdownNumbersGame" when ordering the pairs.

* First class Functions: Functions are treated like data and passed into other functions as attributes in the code this helps with keeping the code clean and helps to avoid state, in the "solveCountdownNumbersGame" function, the "eval_rpn(c)" function is passed into the "displayCalculation" function as an attribute to avoid creating a state, this reduces any possible negative side effects to disrupt the games flow.


## References

***

 * Numbers Game https://wiki.apterous.org/Numbers_game
 * Python Functional Programming https://docs.python.org/3/howto/functional.html
 * Matthew Tyson "What is functional programming? Apractical guide" https://www.infoworld.com/article/3613715/what-is-functional-programming-a-practical-guide.html
 * Computerphile "Functional Programming & Haskell" https://www.youtube.com/watch?v=LnX3B9oaKzw
 * Hitesh Choudhary "What is functional Programming" https://www.youtube.com/watch?v=dAPL7MQGjyM

***

## End